In [1]:
%load_ext autoreload
%autoreload 2

import os, sys, shutil, bz2, copy, warnings
import pandas as pd
pd.set_option('display.max_columns', 50)
# Numpy will complain about subnormals if python
# is compiled with the -ffast-math compiler flag:
# https://github.com/clearlinux/distribution/issues/2809
with warnings.catch_warnings():
    warnings.simplefilter("ignore", message="The value of the smallest subnormal for")
    import numpy as np

/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sulab/anaconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Navigate back to NBdir in case of re-running a code block:
if not 'NBdir' in globals():
    NBdir = os.getcwd()
print('Notebook is in: {}'.format(NBdir))
os.chdir(NBdir)  # If you changed the current working dir, this will take you back to the notebook dir.

# Define the path to the repo folder.
# Change if necessary.
homedir = '/'.join(NBdir.split('/')[0:-2])
print('Repo is in: {}'.format(homedir))
sys.path.insert(1, homedir)
from src.misc import index_to_sample_df, read_tRNAdb_info, sample_df_to_dict
from src.read_processing import AR_merge, BC_split, UMI_trim
from src.alignment import SWIPE_align
from src.stats_collection import STATS_collection

# These are default folder names for data and raw fastq files
# relative to the folder in which this notebook is in:
data_dir = 'data'
seq_dir = 'raw_fastq'

# These folder names are used in subsequent processing steps
# to dump data. Best to not change:
AdapterRemoval_dir = 'AdapterRemoval'
BC_dir = 'BC_split'
UMI_dir = 'UMI_trimmed'
align_dir = 'SWalign'
stats_dir = 'stats_collection'
tRNA_database = dict()
tRNA_database['human'] = '{}/tRNA_database/human/hg38-tRNAs.fa'.format(homedir)
tRNA_database_masked = dict()
tRNA_database_masked['human'] = '{}/tRNA_database_masked/human/human-tRNAs.fa'.format(homedir)
# Read information (length, codon etc) of tRNAs into dictionary:
tRNA_data = read_tRNAdb_info(tRNA_database)
SWIPE_score_mat = '{}/utils/nuc_score-matrix.txt'.format(homedir)
SWIPE_score_mat2 = '{}/utils/nuc_score-matrix_2.txt'.format(homedir) # For masked reference sequences
# tRNA sequencing yields many duplicated reads.
# Adding these commonly seen sequences to a list prevents duplicated alignment:
common_seqs = '{}/utils/common-seqs.fasta.bz2'.format(homedir)

# Define minimum read length based on minimum insert size:
MIN_INSERT_LEN = 10
UMI_LEN = 10
BC_MAX_LEN = 19
MIN_READ_LEN = MIN_INSERT_LEN + UMI_LEN + BC_MAX_LEN
print('Using minimum read length: {} (after merge)'.format(MIN_READ_LEN))

# Read index information:
index_list_fnam = 'index_list.xlsx'
index_df = pd.read_excel('{}/utils/{}'.format(homedir, index_list_fnam))

# Read sample list:
sample_list_fnam = 'sample_list.xlsx'
sample_df = pd.read_excel('{}/{}'.format(NBdir, sample_list_fnam))
# Add barcode sequences:
sample_df = index_to_sample_df(sample_df, index_df)
# Read elementary info (replicate, barcode, species)
# for each unique sample name into a dictionary:
sample_dict = sample_df_to_dict(sample_df)
# Get filenames from the sample information:
inp_file_df = sample_df[['fastq_mate1_filename', 'fastq_mate2_filename', 'P5_index', 'P7_index', 'P5_index_seq', 'P7_index_seq']].copy().drop_duplicates().reset_index(drop=True)
# Make a dictionary with paths used for data processing:
dir_dict = dict(NBdir = NBdir,
                data_dir = data_dir,
                seq_dir = seq_dir,
                AdapterRemoval_dir = AdapterRemoval_dir,
                BC_dir = BC_dir,
                UMI_dir = UMI_dir,
                align_dir = align_dir,
                stats_dir = stats_dir)

Notebook is in: /home/sulab/tRNA-charge-seq/projects/Ser-Pro_anno
Repo is in: /home/sulab/tRNA-charge-seq
Using minimum read length: 39 (after merge)


# Alignment effect on reads mapping to Ser/Pro codons
Process a set of samples covering all adapter barcodes with/without using a masked reference for alignment.
Gather alignment statistics from each case with the purpose of showing the effect of the alignment method on the mapping to Ser/Pro codons.

In [ ]:
# Run AdapterRemoval:
AR_obj = AR_merge(dir_dict, inp_file_df, MIN_READ_LEN, overwrite_dir=True, AR_threads=12)
inp_file_df = AR_obj.run_parallel(n_jobs=4, overwrite=True)

# Split files based on barcodes:
BCsplit_obj = BC_split(dir_dict, sample_df, inp_file_df, overwrite_dir=True)
sample_df, inp_file_df = BCsplit_obj.run_parallel(n_jobs=9, load_previous=False)

# Trim UMIs:
UMItrim_obj = UMI_trim(dir_dict, sample_df, overwrite_dir=True)
sample_df = UMItrim_obj.run_parallel(n_jobs=9, load_previous=False)
sample_df.head(3)

In [7]:
# Align reads to unmasked reference tRNAs:
align_obj = SWIPE_align(dir_dict, tRNA_database, sample_df, SWIPE_score_mat2, \
                        gap_penalty=6, extension_penalty=3, min_score_align=15, \
                        common_seqs=common_seqs, overwrite_dir=True)
sample_df = align_obj.run_parallel(n_jobs=5, overwrite=False, load_previous=False)

# Collect alignment statistics:
stats_obj = STATS_collection(dir_dict, tRNA_data, sample_df, common_seqs=common_seqs, \
                             overwrite_dir=True)
stats_df = stats_obj.run_parallel(n_jobs=9, load_previous=False)
sample_df.head(3)

Using common sequences to prevent duplicated alignment.
Running Swipe on:  100p3_v2  0m_p1_v2  0m_p3_v2  0m_p2_v2  100p1_v1  0m_p4_v2  85p2_v1  85p4_v1  85p1_v1  85p2_v2  85p1_v2  85p3_v2  85p4_v2  70p1_v1  70p3_v2  70p2_v2  70p1_v2  70p4_v2  70p5_v2  55p1_v1  55p3_v2  55p1_v2  55p2_v2  55p3_v1  55p4_v2  55p5_v2  40p4_v1  40p2_v1  40p1_v1  40p1_v2  40p2_v2  40p3_v2  40p4_v2  25p2_v2  25p1_v2  25p1_v1  25p6_v2  10p1_v1  25p3_v2  25p4_v2  25p5_v2  10p3_v1  10p2_v2  10p1_v2  10p3_v2  10p4_v2  10p5_v2  0p1_v1  0p2_v1  0p4_v1  0p4_v2  0p1_v2  0p_90m2_v1  0p_90m1_v1  0p2_v2  0p3_v2  4m_1  0p_90m3_v1  0p_90m4_v1  8m_1  0m_1  32m_1  16m_1  1h_1  8h_1  40h_1  4h_1  16h_1  2h_1  40h_NoOx_1  8m_2  4m_2  32m_2  0m_2  16m_2  1h_2  8h_2  2h_2  4h_2  40h_2  16h_2  40h_NoOx_2  4m_3  0m_3  16m_3  1h_3  8m_3  32m_3  2h_3  4h_3  8h_3  40h_NoOx_3  40h_3  16h_3  0m_4  4m_4  16m_4  8m_4  1h_4  32m_4  2h_4  4h_4  8h_4  16h_4  40h_NoOx_4  40h_4  8h_p1  8h_p2  8h_p4  8h_p5  8h_p6  8h_p3  8h_p7  8h_p8  8h_p9  L

,sample_name_unique,sample_name,replicate,fastq_mate1_filename,fastq_mate2_filename,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_mapped,percent_single_annotation,percent_multiple_annotation,percent_multiple_codons,Mapping_percent
0,100p1_v1,100p,1,2022-12-28/P1_R1.fastq.bz2,2022-12-28/P1_R2.fastq.bz2,D501,D701,l1Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,ATTACTCG,GGCTGCCATGCGACTA,4882120,223691,4625330,4849021,99.322036,95.386883,4801306,515441,524232.741440,98.344695,98.322932,4774581.0,79.221821,20.778179,1.408794,99.443381
1,100p3_v2,100p,3,2022-12-28/P3_R1.fastq.bz2,2022-12-28/P3_R2.fastq.bz2,D501,D703,l2Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,CGCTCATT,GGCTGCCATGCTGTCACG,3002197,149772,2828518,2978290,99.203683,94.971208,2951336,492419,522405.107612,98.305874,94.259990,2918101.0,76.457909,23.542091,2.047804,98.873900
2,0m_p1_v2,100p_0m,1,2023-02-28/P4_R1.fastq.bz2,2023-02-28/P4_R2.fastq.bz2,D502,D701,l5Sp,human,Charge-titration,Percent charge,100p_0m,2,GCCTCTAT,ATTACTCG,GGCTGCCATGCAACGCATC,2989860,163356,2798679,2962035,99.069354,94.485008,2931995,494668,522334.350496,98.064625,94.703325,2895784.0,74.423127,25.576873,2.288948,98.764971


In [ ]:
# Make new folders for the masked alignment/stats:
dir_dict_masked = copy.deepcopy(dir_dict)
dir_dict_masked['align_dir'] = 'SWalign_masked'
dir_dict_masked['stats_dir'] = 'stats_collection_masked'

# Align reads to unmasked reference tRNAs:
align_obj = SWIPE_align(dir_dict_masked, tRNA_database_masked, sample_df, SWIPE_score_mat2, \
                        gap_penalty=6, extension_penalty=3, min_score_align=15, \
                        common_seqs=common_seqs, overwrite_dir=True)
sample_df = align_obj.run_parallel(n_jobs=5, overwrite=False, load_previous=False)

# Collect alignment statistics:
stats_obj = STATS_collection(dir_dict_masked, tRNA_data, sample_df, common_seqs=common_seqs, \
                             overwrite_dir=True)
stats_df = stats_obj.run_parallel(n_jobs=9, load_previous=False)
sample_df.head(3)